In [2]:
import requests
import pandas as pd

In [4]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/IFS/M.GB.PMP_IX'

data = (requests.get(f'{url}{key}').json()
        ['CompactData']['DataSet']['Series'])

In [32]:
# with iteration
d1 = pd.DataFrame([{'date': pd.to_datetime(i['@TIME_PERIOD']), 
                    'value': float(i['@OBS_VALUE'])} 
                   for i in data['Obs']]).set_index('date')

# array-wise
d2 = (pd.DataFrame(data['Obs'])
        .rename({'@TIME_PERIOD': 'date', '@OBS_VALUE': 'value'}, axis=1)
        .astype({'date': 'datetime64[ns]', 'value': 'float'})
        .set_index('date'))

In [34]:
d2

,value
date,
1980-01-01,45.219100
1980-02-01,45.790460
1980-03-01,46.443444
1980-04-01,46.525067
1980-05-01,46.851558
...,...
2019-02-01,104.403131
2019-03-01,103.718200
2019-04-01,105.577299


In [25]:
(pd.DataFrame(data['Obs'])
   .rename({'@TIME_PERIOD': 'date', 
            '@OBS_VALUE': 'value'}, axis=1)
   .astype({'date': 'datetime64[ns]', 
            'value': 'float'})
   .set_index('date'))

,value
date,
1980-01-01,45.219100
1980-02-01,45.790460
1980-03-01,46.443444
1980-04-01,46.525067
1980-05-01,46.851558
...,...
2019-02-01,104.403131
2019-03-01,103.718200
2019-04-01,105.577299


NameError: name 'df' is not defined